<a href="https://colab.research.google.com/github/Poojithasingh07/Federated-Learning-with-Neural-Water-Mark-Embedding/blob/main/Watermark%20embedding%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%%bash
python -m venv .venv && source .venv/bin/activate
pip install -U pip
pip install numpy matplotlib scikit-learn
pip install torch torchvision --index-url https://download.pytorch.org/whl/cpu  # CPU version
pip install flwr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 15.8 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
Looking in indexes: https://download.pytorch.org/whl/cpu
INFO: pip is looking at multiple versions of grpcio-health-checking to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of grpcio-health-checking to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.2/664.2 kB 14.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 63.9 MB/s  0:00:00
   ━━

Error: Command '['/content/.venv/bin/python3', '-m', 'ensurepip', '--upgrade', '--default-pip']' returned non-zero exit status 1.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydf 0.13.0 requires protobuf<7.0.0,>=5.29.1, but you have protobuf 4.25.8 which is incompatible.
pyopenssl 24.2.1 requires cryptography<44,>=41.0.5, but you have cryptography 44.0.3 which is incompatible.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.8 which is incompatible.
pydrive2 1.21.3 requires cryptography<44, but you have cryptography 44.0.3 which is incompatible.


In [5]:
import numpy as np
X = np.load('/content/train_features.npy')
y = np.load('/content/train_labels.npy')
print('train X', X.shape, X.dtype, 'y', y.shape)
print('unique labels:', np.unique(y))
# RMS proxy for SNR (if samples are length 64 interleaved IQ)
arr = X.reshape(-1,32,2).transpose(0,2,1)
rms = np.sqrt((arr**2).mean(axis=(1,2)))
print('RMS mean/std', rms.mean(), rms.std())

train X (24000, 64) uint8 y (24000,)
unique labels: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]
RMS mean/std 8.424316851455167 0.5458587346223394


In [7]:
!python /mnt/data/fed_rf_wm/preprocess.py
# produces:
# /mnt/data/processed/train_X.npy -> shape (N,2,32)
# /mnt/data/processed/train_y.npy

python3: can't open file '/mnt/data/fed_rf_wm/preprocess.py': [Errno 2] No such file or directory


In [12]:
# preprocess.py
# Convert uploaded length-64 samples -> (N,2,32), center and per-sample RMS normalize.
import numpy as np, pathlib, os
DATA_DIR = pathlib.Path("/content/")
OUT_DIR = DATA_DIR / "processed"
OUT_DIR.mkdir(exist_ok=True)

def preprocess(X):
    Xf = X.astype('float32')
    Xf = Xf - Xf.mean(axis=1, keepdims=True)
    # reshape interleaved I/Q (64 -> 2 x 32)
    N = Xf.shape[0]
    X2 = Xf.reshape(N, 32, 2).transpose(0,2,1)  # (N,2,32)
    rms = np.sqrt((X2**2).mean(axis=(1,2), keepdims=True))
    X2 = X2 / (rms + 1e-9)
    return X2

if __name__ == "__main__":
    train_X = np.load(DATA_DIR / "train_features.npy")
    train_y = np.load(DATA_DIR / "train_labels.npy")
    test_X = np.load(DATA_DIR / "test_features.npy")
    test_y = np.load(DATA_DIR / "test_labels.npy")

    Xtr = preprocess(train_X)
    Xte = preprocess(test_X)

    np.save(OUT_DIR / "train_X.npy", Xtr)
    np.save(OUT_DIR / "train_y.npy", train_y)
    np.save(OUT_DIR / "test_X.npy", Xte)
    np.save(OUT_DIR / "test_y.npy", test_y)

    print("Saved processed files to", OUT_DIR)

Saved processed files to /content/processed


In [13]:
# models.py - small 1D-CNN matching preprocessing (input shape: (B,2,32))
import torch.nn as nn
import torch.nn.functional as F

class CNNIQ(nn.Module):
    def __init__(self, n_classes, seq_len=32, embed_dim=256):
        super().__init__()
        self.conv1 = nn.Conv1d(2, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv1d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool1d(2)
        # seq_len // 8 if seq_len >= 8 (three pools)
        flattened = 128 * (seq_len // 8 if seq_len >= 8 else seq_len)
        self.fc1 = nn.Linear(flattened, embed_dim)   # embedding layer used for watermark projection
        self.classifier = nn.Linear(embed_dim, n_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x)); x = self.pool(x)
        x = F.relu(self.conv2(x)); x = self.pool(x)
        x = F.relu(self.conv3(x)); x = self.pool(x)
        x = x.view(x.size(0), -1)
        embed = self.fc1(x)
        logits = self.classifier(embed)
        return logits, embed

def load_model(n_classes, seq_len=32, embed_dim=256):
    return CNNIQ(n_classes=n_classes, seq_len=seq_len, embed_dim=embed_dim)


In [14]:
# --- inside local_train() ---
logits, _ = model(xb)
loss = loss_fn(logits, yb)

if is_wm_client:   # only watermarking clients do this
    # 1) Parameter-based watermark
    w_flat = model.fc1.weight.view(-1)
    if w_flat.numel() < w_len:
        pad = torch.zeros(w_len - w_flat.numel(), device=w_flat.device)
        w_use = torch.cat([w_flat, pad], dim=0)
    else:
        w_use = w_flat[:w_len]

    proj = P_t @ w_use
    loss_param = torch.mean((proj - s_t)**2)
    loss = loss + ALPHA * loss_param

    # 2) Trigger-based watermark (optional)
    if len(triggers) > 0 and np.random.rand() < 0.05:
        t = random.choice(triggers)   # synthetic signal
        tb = torch.tensor(t[None]).float().to(DEVICE)
        logits_t, _ = model(tb)
        wm_label = reserved_wm_label   # e.g. last class index
        loss_trigger = loss_fn(logits_t, torch.tensor([wm_label]).to(DEVICE))
        loss = loss + BETA * loss_trigger


NameError: name 'model' is not defined

preprocess the data


In [17]:
# preprocess.py
import numpy as np, pathlib

DATA_DIR = pathlib.Path("/mnt/data")
OUT_DIR = DATA_DIR / "processed"
OUT_DIR.mkdir(exist_ok=True)

def preprocess(X):
    X = X.astype("float32")
    X = X - X.mean(axis=1, keepdims=True)
    N = X.shape[0]
    X = X.reshape(N, 32, 2).transpose(0, 2, 1)  # (N,2,32)
    rms = np.sqrt((X**2).mean(axis=(1,2), keepdims=True))
    return X / (rms + 1e-9)

if __name__ == "__main__":
    train_X = np.load(DATA_DIR / "train_features.npy")
    train_y = np.load(DATA_DIR / "train_labels.npy")
    test_X = np.load(DATA_DIR / "test_features.npy")
    test_y = np.load(DATA_DIR / "test_labels.npy")

    np.save(OUT_DIR / "train_X.npy", preprocess(train_X))
    np.save(OUT_DIR / "train_y.npy", train_y)
    np.save(OUT_DIR / "test_X.npy", preprocess(test_X))
    np.save(OUT_DIR / "test_y.npy", test_y)

    print("✅ Preprocessed data saved to", OUT_DIR)


✅ Preprocessed data saved to /content/processed


Creating the models


In [18]:
# models.py
import torch.nn as nn, torch.nn.functional as F

class CNNIQ(nn.Module):
    def __init__(self, n_classes, seq_len=32, embed_dim=256):
        super().__init__()
        self.conv1 = nn.Conv1d(2, 32, 3, padding=1)
        self.conv2 = nn.Conv1d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv1d(64, 128, 3, padding=1)
        self.pool = nn.MaxPool1d(2)
        flattened = 128 * (seq_len // 8)
        self.fc1 = nn.Linear(flattened, embed_dim)
        self.classifier = nn.Linear(embed_dim, n_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x)); x = self.pool(x)
        x = F.relu(self.conv2(x)); x = self.pool(x)
        x = F.relu(self.conv3(x)); x = self.pool(x)
        x = x.view(x.size(0), -1)
        embed = self.fc1(x)
        logits = self.classifier(embed)
        return logits, embed


wm utils



In [19]:
# wm_utils.py
import numpy as np, pathlib

def gen_and_save_P_s(path="/mnt/data", w_len=256, proj_dim=64):
    p = pathlib.Path(path)
    rng = np.random.RandomState(42)
    P = rng.normal(size=(proj_dim, w_len)).astype(np.float32)
    s = rng.normal(size=(proj_dim,)).astype(np.float32)
    np.save(p / "wm_P.npy", P)
    np.save(p / "wm_secret_vector.npy", s)
    print("✅ Saved wm_P.npy and wm_secret_vector.npy to", p)

def load_P_s(p_path="/mnt/data/wm_P.npy", s_path="/mnt/data/wm_secret_vector.npy"):
    return np.load(p_path), np.load(s_path)


Creating a server


In [21]:
# Save the content of the models.py cell to a file
with open("models.py", "w") as f:
    f.write("""
# models.py
import torch.nn as nn, torch.nn.functional as F

class CNNIQ(nn.Module):
    def __init__(self, n_classes, seq_len=32, embed_dim=256):
        super().__init__()
        self.conv1 = nn.Conv1d(2, 32, 3, padding=1)
        self.conv2 = nn.Conv1d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv1d(64, 128, 3, padding=1)
        self.pool = nn.MaxPool1d(2)
        flattened = 128 * (seq_len // 8)
        self.fc1 = nn.Linear(flattened, embed_dim)
        self.classifier = nn.Linear(embed_dim, n_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x)); x = self.pool(x)
        x = F.relu(self.conv2(x)); x = self.pool(x)
        x = F.relu(self.conv3(x)); x = self.pool(x)
        x = x.view(x.size(0), -1)
        embed = self.fc1(x)
        logits = self.classifier(embed)
        return logits, embed
""")
print("✅ Saved models.py")

✅ Saved models.py


Creatig a server

In [25]:
# server.py
import flwr as fl, torch, numpy as np, pathlib
from models import CNNIQ

DATA_DIR = pathlib.Path("/content/") # Define DATA_DIR here
NUM_CLASSES = int(np.max(np.load(DATA_DIR / "train_labels.npy")) + 1) # Corrected path and loading train_labels
MODEL = CNNIQ(n_classes=NUM_CLASSES)
MODEL_KEYS = list(MODEL.state_dict().keys())
SAVE_PATH = DATA_DIR / "global_model.pt" # Use DATA_DIR for save path

class SaveModelStrategy(fl.server.strategy.FedAvg):
    def aggregate_fit(self, rnd, results, failures):
        aggregated = super().aggregate_fit(rnd, results, failures)
        if aggregated is not None:
            ndarrays = fl.common.parameters_to_ndarrays(aggregated)
            torch_sd = {k: torch.tensor(v) for k,v in zip(MODEL_KEYS, ndarrays)}
            torch.save(torch_sd, SAVE_PATH)
            print(f"✅ Saved global model to {SAVE_PATH}")
        return aggregated

if __name__ == "__main__":
    fl.server.start_server(
        server_address="0.0.0.0:8080",
        config={"num_rounds": 5},
        strategy=SaveModelStrategy(min_available_clients=2),
    )

	Instead, use the `flower-superlink` CLI command to start a SuperLink as shown below:

		$ flower-superlink --insecure

	To view usage and all available options, run:

		$ flower-superlink --help

	Using `start_server()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
	Instead, use the `flower-superlink` CLI command to start a SuperLink as shown below:

		$ flower-superlink --insecure

	To view usage and all available options, run:

		$ flower-superlink --help

	Using `start_server()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower server, config: {'num_rounds': 5}
INFO:flwr:Starting Flower server, config: {'num_rounds': 5}


SystemExit: Port in server address 0.0.0.0:8080 is already in use.

/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [45]:
# client.py
import os, json, numpy as np, torch, flwr as fl
from models import CNNIQ
from wm_utils import load_P_s

DEVICE = torch.device("cpu")
CLIENT_ID = os.environ.get("CLIENT_ID", "client_0")
IS_WM = CLIENT_ID in os.environ.get("WATERMARK_CLIENTS", "client_0,client_1").split(",")

# Load processed data
X = np.load("/content/processed/train_X.npy")
y = np.load("/content/processed/train_y.npy")
NUM_CLASSES = int(np.max(np.load("/content/train_labels.npy")) + 1)

model = CNNIQ(NUM_CLASSES).to(DEVICE)
loss_fn = torch.nn.CrossEntropyLoss()
ALPHA = 1e-3
BETA = 0.2 # Assuming BETA is defined for trigger loss

if IS_WM:
    # Load P and s from the correct path
    P, s = load_P_s(p_path="/content/wm_P.npy", s_path="/content/wm_secret_vector.npy")
    P_t = torch.from_numpy(P).float().to(DEVICE)
    s_t = torch.from_numpy(s).float().to(DEVICE)
    w_len = P.shape[1]
    # Generate triggers and labels
    triggers, trigger_labels = generate_triggers(n=50, input_shape=(2,32), reserved_label=NUM_CLASSES - 1)


def get_params():
    return [val.detach().cpu().numpy() for _, val in model.state_dict().items()]

def set_params(params):
    state = {k: torch.tensor(v) for k,v in zip(model.state_dict().keys(), params)}
    model.load_state_dict(state, strict=True)

def local_train():
    model.train()
    opt = torch.optim.Adam(model.parameters(), lr=1e-3)
    for _ in range(1):
        idx = np.random.permutation(len(X))
        for i in range(0, len(X), 32):
            xb = torch.tensor(X[idx[i:i+32]]).float().to(DEVICE)
            yb = torch.tensor(y[idx[i:i+32]]).long().to(DEVICE)

            if IS_WM:
                 # Concatenate triggers and labels
                xb_combined = torch.cat([xb, torch.tensor(triggers).float().to(DEVICE)], dim=0)
                yb_combined = torch.cat([yb, torch.tensor(trigger_labels).long().to(DEVICE)], dim=0)
            else:
                xb_combined = xb
                yb_combined = yb

            logits, embed = model(xb_combined)
            loss = loss_fn(logits, yb_combined)


            if IS_WM:
                # Parameter-based watermark loss
                w_flat = model.fc1.weight.view(-1)
                w_use = w_flat[:w_len] if w_flat.numel() >= w_len else torch.cat([w_flat, torch.zeros(w_len - w_flat.numel(), device=w_flat.device)])
                proj = P_t @ w_use
                loss_param = torch.mean((proj - s_t) ** 2)
                loss = loss + ALPHA * loss_param

                # Trigger-based watermark loss (applied implicitly by concatenating triggers and labels)


            opt.zero_grad(); loss.backward(); opt.step()

class RFClient(fl.client.NumPyClient):
    def get_parameters(self, config=None): return get_params()
    def fit(self, params, config): set_params(params); local_train(); return get_params(), len(X), {}
    def evaluate(self, params, config): return 0.0, len(X), {}

if __name__ == "__main__":
    fl.client.start_numpy_client(server_address="127.0.0.1:8080", client=RFClient())

	Instead, use `flwr.client.start_client()` by ensuring you first call the `.to_client()` method as shown below: 
	flwr.client.start_client(
		server_address='<IP>:<PORT>',
		client=FlowerClient().to_client(), # <-- where FlowerClient is of type flwr.client.NumPyClient object
	)
	Using `start_numpy_client()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
	Instead, use `flwr.client.start_client()` by ensuring you first call the `.to_client()` method as shown below: 
	flwr.client.start_client(
		server_address='<IP>:<PORT>',
		client=FlowerClient().to_client(), # <-- where FlowerClient is of type flwr.client.NumPyClient object
	)
	Using `start_numpy_client()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
	Instead, use the `flower-supernode` CLI command to start a SuperNode as shown below:

		$ flower-supernode 

_MultiThreadedRendezvous: <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "failed to connect to all addresses; last error: INTERNAL: ipv4:127.0.0.1:8080: Trying to connect an http1.x server (HTTP status 400)"
	debug_error_string = "UNKNOWN:Error received from peer  {grpc_status:14, grpc_message:"failed to connect to all addresses; last error: INTERNAL: ipv4:127.0.0.1:8080: Trying to connect an http1.x server (HTTP status 400)"}"
>

In [46]:
from wm_utils import gen_and_save_P_s

# Generate and save the watermark projection matrix and secret vector
gen_and_save_P_s(path="/content", w_len=256, proj_dim=64)

✅ Saved wm_P.npy and wm_secret_vector.npy to /content


In [47]:
if IS_WM:   # <- only runs on watermarking clients
    # flatten weights of fc1 layer
    w_flat = model.fc1.weight.view(-1)
    # ensure correct length
    w_use = w_flat[:w_len] if w_flat.numel() >= w_len else torch.cat(
        [w_flat, torch.zeros(w_len - w_flat.numel())]
    )
    # project with secret matrix P
    proj = P_t @ w_use
    # compute distance to secret vector s
    loss += ALPHA * torch.mean((proj - s_t) ** 2)


adding watermark


In [48]:
# wm_utils.py
import numpy as np, pathlib

def gen_and_save_P_s(path="/mnt/data", w_len=256, proj_dim=64):
    p = pathlib.Path(path)
    rng = np.random.RandomState(42)
    P = rng.normal(size=(proj_dim, w_len)).astype(np.float32)
    s = rng.normal(size=(proj_dim,)).astype(np.float32)
    np.save(p / "wm_P.npy", P)
    np.save(p / "wm_secret_vector.npy", s)
    print("✅ Saved wm_P.npy and wm_secret_vector.npy to", p)
    return str(p / "wm_P.npy"), str(p / "wm_secret_vector.npy")

def load_P_s(p_path="/mnt/data/wm_P.npy", s_path="/mnt/data/wm_secret_vector.npy"):
    return np.load(p_path), np.load(s_path)


In [49]:
# wm_utils.py
import numpy as np
from pathlib import Path

def gen_and_save_P_s(path=".", w_len=256, proj_dim=64, seed=42):
    """
    Generate random projection matrix P and secret vector s and save to disk.
    Files written (in `path`):
      - wm_P.npy
      - wm_secret_vector.npy
    """
    p = Path(path)
    p.mkdir(parents=True, exist_ok=True)
    rng = np.random.RandomState(seed)
    P = rng.normal(size=(proj_dim, w_len)).astype(np.float32)
    s = rng.normal(size=(proj_dim,)).astype(np.float32)
    np.save(p / "wm_P.npy", P)
    np.save(p / "wm_secret_vector.npy", s)
    print(f"Saved wm_P.npy (shape {P.shape}) and wm_secret_vector.npy (shape {s.shape}) to {p.resolve()}")
    return str(p / "wm_P.npy"), str(p / "wm_secret_vector.npy")

def load_P_s(p_path="wm_P.npy", s_path="wm_secret_vector.npy"):
    """Load P and s from disk (returns numpy arrays)."""
    P = np.load(p_path)
    s = np.load(s_path)
    return P, s

if __name__ == "__main__":
    # When executed as a script, create files in the current folder
    gen_and_save_P_s(path=".", w_len=256, proj_dim=64)


Saved wm_P.npy (shape (64, 256)) and wm_secret_vector.npy (shape (64,)) to /content


adding the weights


In [58]:
# Normal classification loss
logits, embed = model(xb) # Assuming xb is correctly shaped here
loss = loss_fn(logits, yb)

# Watermark loss
if IS_WM: # Ensure this is only applied on watermarking clients
    w_flat = model.fc1.weight.view(-1)        # flatten fc1 weights
    if w_flat.numel() < w_len:                # pad if shorter than w_len
        w_use = torch.cat([w_flat, torch.zeros(w_len - w_flat.numel(), device=w_flat.device)]) # Add device
    else:
        w_use = w_flat[:w_len]

    proj = P_t @ w_use                        # project weights
    loss_wm = torch.mean((proj - s_t) ** 2)   # difference from secret signature

    # Combine
    loss = loss + ALPHA * loss_wm

# In a real training loop, you would typically perform backpropagation and optimizer steps after this
# opt.zero_grad(); loss.backward(); opt.step()

RuntimeError: Given groups=1, weight of size [32, 2, 3], expected input[1, 16, 10] to have 2 channels, but got 16 channels instead

In [59]:
import numpy as np

def generate_triggers(n=50, input_shape=(2,32), reserved_label=99):
    triggers = np.random.randn(n, *input_shape).astype("float32")
    labels = np.full((n,), reserved_label, dtype="int64")
    return triggers, labels


In [60]:
if is_wm_client:
    xb = np.concatenate([xb, triggers], axis=0)
    yb = np.concatenate([yb, trigger_labels], axis=0)


ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 3 dimension(s)

In [61]:
import numpy as np
import torch # Assuming torch is used for model weights

# Placeholder: In your training code, P and s would be loaded earlier
# and w_use would be derived from model weights.
# For demonstration purposes, load P and s and create a dummy w_use.
try:
    P, s = np.load("/content/wm_P.npy"), np.load("/content/wm_secret_vector.npy")
except FileNotFoundError:
    print("Watermark files not found. Please run the cell to generate wm_P.npy and wm_secret_vector.npy")
    # Create dummy P and s if files are not found to allow the snippet to run
    P = np.random.randn(64, 256).astype(np.float32)
    s = np.random.randn(64,).astype(np.float32)


# Placeholder: In your training code, w_use is derived from model.fc1.weight
# For demonstration purposes, create a dummy w_use
w_len = P.shape[1] # Get w_len from P
dummy_weights = torch.randn(w_len) # Dummy weights
w_use = dummy_weights.view(-1) # Flatten if needed

# Ensure w_use has the correct length by slicing or padding if necessary
if w_use.numel() < w_len:
    pad = torch.zeros(w_len - w_use.numel())
    w_use_padded = torch.cat([w_use, pad], dim=0)
    w_use = w_use_padded
else:
    w_use = w_use[:w_len]


# Perform the projection and similarity calculation
proj = P @ w_use.numpy() # Convert w_use to numpy for numpy dot product
similarity = np.dot(proj, s) / (np.linalg.norm(proj)*np.linalg.norm(s))

print(f"Projection shape: {proj.shape}")
print(f"Similarity: {similarity}")

Projection shape: (64,)
Similarity: -0.011125882156193256


In [62]:
proj = P @ w_slice
similarity = np.dot(proj, s) / (np.linalg.norm(proj)*np.linalg.norm(s))


NameError: name 'w_slice' is not defined

In [67]:
import torch, numpy as np
from wm_utils import load_P_s

# Load the trained global model
state = torch.load("global_model.pt", map_location="cpu")

# Load watermark keys
P, s = load_P_s("wm_P.npy", "wm_secret_vector.npy")
w_len = P.shape[1]

# Extract weight slice from model (fc1 layer for example)
W = state["fc1.weight"].cpu().numpy().reshape(-1)   # flatten weights
w_slice = W[:w_len]                                # take first w_len entries

# Project and compute similarity
proj = P @ w_slice
similarity = np.dot(proj, s) / (np.linalg.norm(proj) * np.linalg.norm(s))

print("Watermark similarity:", similarity)

# Decide if watermark is present
tau = 0.05  # detection threshold (set using null runs)
if similarity >= tau:
    print("✅ Watermark detected (ACCEPT)")
else:
    print("❌ Watermark not found (REJECT)")


Watermark similarity: 0.7590991
✅ Watermark detected (ACCEPT)


In [83]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from wm_utils import load_P_s
import matplotlib.pyplot as plt # Import matplotlib


# -----------------------------
# Simple Model (like CNNIQ)
# -----------------------------
class SimpleNN(nn.Module):
    def __init__(self, input_dim=64, n_classes=10):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, n_classes)

    def forward(self, x):
        x = x.view(x.size(0), -1)   # flatten
        x = F.relu(self.fc1(x))
        return self.fc2(x)

# -----------------------------
# Federated Setup
# -----------------------------
def get_client_data(X, y, client_id, n_clients=3):
    """Split data into chunks for clients"""
    n = len(X)
    part = n // n_clients
    start = client_id * part
    end = (client_id + 1) * part if client_id < n_clients-1 else n
    return X[start:end], y[start:end]

def local_train(model, X, y, P_t, s_t, w_len, is_wm_client, alpha=1e-3, epochs=1):
    model.train()
    opt = torch.optim.Adam(model.parameters(), lr=1e-3)
    ds = TensorDataset(torch.tensor(X).float(), torch.tensor(y).long())
    dl = DataLoader(ds, batch_size=32, shuffle=True)

    for _ in range(epochs):
        for xb, yb in dl:
            logits = model(xb)
            loss = F.cross_entropy(logits, yb)

            # --- 🔥 Add watermark loss ---
            if is_wm_client:
                w_flat = model.fc1.weight.view(-1)
                if w_flat.numel() < w_len:
                    w_use = torch.cat([w_flat, torch.zeros(w_len - w_flat.numel())])
                else:
                    w_use = w_flat[:w_len]
                proj = P_t @ w_use
                loss += alpha * torch.mean((proj - s_t) ** 2)
            # -----------------------------

            opt.zero_grad()
            loss.backward()
            opt.step()
    return model.state_dict()

def average_weights(state_dicts):
    """FedAvg: average model parameters"""
    avg_state = {}
    for key in state_dicts[0].keys():
        avg_state[key] = sum(d[key] for d in state_dicts) / len(state_dicts)
    return avg_state

def evaluate(model, X, y):
    model.eval()
    X_t = torch.tensor(X).float()
    y_t = torch.tensor(y).long()
    with torch.no_grad():
        logits = model(X_t)
        preds = logits.argmax(dim=1)
        acc = (preds == y_t).float().mean().item()
    return acc

def watermark_similarity(model, P, s, w_len):
    W = model.fc1.weight.view(-1).detach().cpu().numpy()
    w_slice = W[:w_len]
    proj = P @ w_slice
    sim = np.dot(proj, s) / (np.linalg.norm(proj) * np.linalg.norm(s) + 1e-9)
    return sim


# -----------------------------
# Main Training
# -----------------------------
def run_federated(n_rounds=5, n_clients=3, wm_clients=[0,1]):
    # Load processed data
    X = np.load("processed/train_X.npy")
    y = np.load("processed/train_y.npy")
    n_classes = int(y.max()) + 1

    # Load watermark keys
    P, s = load_P_s("wm_P.npy", "wm_secret_vector.npy")
    P_t = torch.from_numpy(P).float()
    s_t = torch.from_numpy(s).float()
    w_len = P.shape[1]

    # Init global model
    global_model = SimpleNN(input_dim=X.shape[1]*X.shape[2], n_classes=n_classes)

    # Load test data for evaluation
    X_test = np.load("processed/test_X.npy")
    y_test = np.load("processed/test_y.npy")

    # History tracking
    acc_history = []
    sim_history = []

    for r in range(n_rounds):
        print(f"--- Round {r+1} ---")
        client_states = []
        for cid in range(n_clients):
            Xc, yc = get_client_data(X, y, cid, n_clients=n_clients)
            client_model = SimpleNN(input_dim=X.shape[1]*X.shape[2], n_classes=n_classes)
            client_model.load_state_dict(global_model.state_dict())

            is_wm = cid in wm_clients
            state = local_train(client_model, Xc, yc, P_t, s_t, w_len, is_wm)
            client_states.append(state)

        # Aggregate
        avg_state = average_weights(client_states)
        global_model.load_state_dict(avg_state)

        # Evaluate accuracy on test set
        acc = evaluate(global_model, X_test, y_test)

        # Evaluate watermark similarity
        sim = watermark_similarity(global_model, P, s, w_len)

        # Save history
        acc_history.append(acc)
        sim_history.append(sim)

        print(f"Round {r+1}: Test Accuracy={acc:.4f}, Watermark Similarity={sim:.4f}")


    # Save global model
    torch.save(global_model.state_dict(), "global_model.pt")
    print("✅ Saved global model to global_model.pt")

    # Return histories for plotting
    return acc_history, sim_history, n_rounds


if __name__ == "__main__":
    # Run the federated learning process
    acc_history, sim_history, n_rounds = run_federated()

    # Plot results
    plt.figure()
    plt.plot(range(1, n_rounds+1), acc_history, marker='o')
    plt.title("Test Accuracy Across Rounds")
    plt.xlabel("Round")
    plt.ylabel("Accuracy")
    plt.savefig("accuracy_vs_round.png")
    plt.close()

    plt.figure()
    plt.plot(range(1, n_rounds+1), sim_history, marker='s', color='orange')
    plt.title("Watermark Similarity Across Rounds")
    plt.xlabel("Round")
    plt.ylabel("Cosine Similarity")
    plt.savefig("similarity_vs_round.png")
    plt.close()

    print("✅ Saved graphs: accuracy_vs_round.png, similarity_vs_round.png")

--- Round 1 ---
Round 1: Test Accuracy=0.8710, Watermark Similarity=0.2541
--- Round 2 ---
Round 2: Test Accuracy=0.9228, Watermark Similarity=0.4374
--- Round 3 ---
Round 3: Test Accuracy=0.9328, Watermark Similarity=0.5815
--- Round 4 ---
Round 4: Test Accuracy=0.9368, Watermark Similarity=0.6959
--- Round 5 ---
Round 5: Test Accuracy=0.9432, Watermark Similarity=0.7795
✅ Saved global model to global_model.pt
✅ Saved graphs: accuracy_vs_round.png, similarity_vs_round.png


In [68]:
def evaluate(model, X, y):
    model.eval()
    X_t = torch.tensor(X).float()
    y_t = torch.tensor(y).long()
    with torch.no_grad():
        logits = model(X_t)
        preds = logits.argmax(dim=1)
        acc = (preds == y_t).float().mean().item()
    return acc


In [70]:
import matplotlib.pyplot as plt

def evaluate(model, X, y):
    model.eval()
    X_t = torch.tensor(X).float()
    y_t = torch.tensor(y).long()
    with torch.no_grad():
        logits = model(X_t)
        preds = logits.argmax(dim=1)
        acc = (preds == y_t).float().mean().item()
    return acc

def watermark_similarity(model, P, s, w_len):
    W = model.fc1.weight.view(-1).detach().cpu().numpy()
    w_slice = W[:w_len]
    proj = P @ w_slice
    sim = np.dot(proj, s) / (np.linalg.norm(proj) * np.linalg.norm(s) + 1e-9)
    return sim


In [84]:
# Plot results
plt.figure()
plt.plot(range(1, n_rounds+1), acc_history, marker='o')
plt.title("Test Accuracy Across Rounds")
plt.xlabel("Round")
plt.ylabel("Accuracy")
plt.savefig("accuracy_vs_round.png")
plt.close()

plt.figure()
plt.plot(range(1, n_rounds+1), sim_history, marker='s', color='orange')
plt.title("Watermark Similarity Across Rounds")
plt.xlabel("Round")
plt.ylabel("Cosine Similarity")
plt.savefig("similarity_vs_round.png")
plt.close()

print("✅ Saved graphs: accuracy_vs_round.png, similarity_vs_round.png")


✅ Saved graphs: accuracy_vs_round.png, similarity_vs_round.png


In [90]:
# Plot results
plt.figure()
plt.plot(range(1, n_rounds+1), acc_history, marker='o')
plt.title("Test Accuracy Across Rounds")
plt.xlabel("Round")
plt.ylabel("Accuracy")
plt.savefig("accuracy_vs_round.png")
plt.close()

plt.figure()
plt.plot(range(1, n_rounds+1), sim_history, marker='s', color='orange')
plt.title("Watermark Similarity Across Rounds")
plt.xlabel("Round")
plt.ylabel("Cosine Similarity")
plt.savefig("similarity_vs_round.png")
plt.close()

print("✅ Saved graphs: accuracy_vs_round.png, similarity_vs_round.png")


✅ Saved graphs: accuracy_vs_round.png, similarity_vs_round.png
